In [29]:
!pip install langchain chromadb tiktoken

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [30]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [31]:
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [32]:
from langchain.schema import Document

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [33]:
docs = [doc1,doc2,doc3,doc4,doc5]

In [34]:
docs

[Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'),
 Document(metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
 Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Che

In [40]:
vector_store = Chroma(
    embedding_function=embedding,
    persist_directory="db/my_chroma_db",
    collection_name="sample"
)

In [41]:
vector_store.add_documents(docs)

['32b3558d-5920-4d98-8252-3bc487f2df00',
 '917992f6-3884-49f4-b2d9-f076529dcc1f',
 '4ccbdddc-0ec3-4663-82a1-ed3e5c86ea88',
 'dfa829b6-337e-4f26-8469-a6bf91954137',
 '5b9bdfcd-9439-449d-b15a-c2e5e40fc7ec']

In [42]:
vector_store.get(include=["embeddings","documents","metadatas"])

{'ids': ['32b3558d-5920-4d98-8252-3bc487f2df00',
  '917992f6-3884-49f4-b2d9-f076529dcc1f',
  '4ccbdddc-0ec3-4663-82a1-ed3e5c86ea88',
  'dfa829b6-337e-4f26-8469-a6bf91954137',
  '5b9bdfcd-9439-449d-b15a-c2e5e40fc7ec'],
 'embeddings': array([[ 0.00994719,  0.06914335, -0.05147116, ..., -0.03543337,
          0.01284809,  0.01248283],
        [ 0.00127741,  0.03129852, -0.02375373, ..., -0.00518364,
         -0.03280607,  0.02737714],
        [-0.10265923,  0.02650806,  0.02271505, ..., -0.0335975 ,
         -0.07984944, -0.01507706],
        [ 0.02123401, -0.02468547, -0.0449437 , ..., -0.10995813,
          0.00572558,  0.09915373],
        [ 0.01873972,  0.04382848, -0.0430425 , ..., -0.07801614,
         -0.07840678, -0.00304191]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [43]:
vector_store.similarity_search(
    query="Who among these are bowler?",
    k=1 # Top result similar to your query
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.')]

In [44]:
vector_store.similarity_search_with_score(
    query="Who among these are bowler?",
    k=1
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.9650975465774536)]

In [45]:
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Mumbai Indians"}
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.865139126777649),
 (Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  1.9243814945220947)]

In [46]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id="32b3558d-5920-4d98-8252-3bc487f2df00", document=updated_doc1)

In [48]:
vector_store.get(include=["embeddings","documents","metadatas"])

{'ids': ['32b3558d-5920-4d98-8252-3bc487f2df00',
  '917992f6-3884-49f4-b2d9-f076529dcc1f',
  '4ccbdddc-0ec3-4663-82a1-ed3e5c86ea88',
  'dfa829b6-337e-4f26-8469-a6bf91954137',
  '5b9bdfcd-9439-449d-b15a-c2e5e40fc7ec'],
 'embeddings': array([[-0.00233743,  0.05902077, -0.04774046, ..., -0.07264048,
          0.00276787, -0.00344092],
        [ 0.00127741,  0.03129852, -0.02375373, ..., -0.00518364,
         -0.03280607,  0.02737714],
        [-0.10265923,  0.02650806,  0.02271505, ..., -0.0335975 ,
         -0.07984944, -0.01507706],
        [ 0.02123401, -0.02468547, -0.0449437 , ..., -0.10995813,
          0.00572558,  0.09915373],
        [ 0.01873972,  0.04382848, -0.0430425 , ..., -0.07801614,
         -0.07840678, -0.00304191]], shape=(5, 384)),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple ce

In [49]:
vector_store.delete(ids=["32b3558d-5920-4d98-8252-3bc487f2df00"])

In [50]:
vector_store.get(include=["embeddings","documents","metadatas"])

{'ids': ['917992f6-3884-49f4-b2d9-f076529dcc1f',
  '4ccbdddc-0ec3-4663-82a1-ed3e5c86ea88',
  'dfa829b6-337e-4f26-8469-a6bf91954137',
  '5b9bdfcd-9439-449d-b15a-c2e5e40fc7ec'],
 'embeddings': array([[ 0.00127741,  0.03129852, -0.02375373, ..., -0.00518364,
         -0.03280607,  0.02737714],
        [-0.10265923,  0.02650806,  0.02271505, ..., -0.0335975 ,
         -0.07984944, -0.01507706],
        [ 0.02123401, -0.02468547, -0.0449437 , ..., -0.10995813,
          0.00572558,  0.09915373],
        [ 0.01873972,  0.04382848, -0.0430425 , ..., -0.07801614,
         -0.07840678, -0.00304191]], shape=(4, 384)),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah is